# 5.7 使用重复元素的网络（VGG）

AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

----
![VGG网络](../img/chapter05/vgg.png)

----
<center>VGG网络</center>

本节介绍VGG，它的名字来源于论文作者所在的实验室Visual Geometry Group [1]。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。

## 5.7.1 VGG块

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用`vgg_block`函数来实现这个基础的VGG块，它可以指定卷积层的数量`num_convs`和输出通道数`num_channels`。

In [11]:
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
import time
import torch
from torch import nn, optim


print(torch.__version__)

1.3.0+cpu


In [12]:
def vgg_block(num_convs, in_channels, out_channels):
    blk = nn.Sequential( nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                         nn.ReLU() )
    for i in range(1, num_convs):
        blk.add_module( str(2*i), nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1) )
        blk.add_module( str(2*i+1), nn.ReLU() )
    blk.add_module( str(num_convs*2 ), nn.MaxPool2d(kernel_size=2, stride=2) )
    return blk

测试一下vgg_blok。☟

In [4]:
test_block = vgg_block(2, 128, 256) # 2个卷积层，输入通道为128，输出通道为256
print(test_block)

Sequential(
  (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


## 5.7.2 VGG网络

与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个`vgg_block`，其超参数由变量`conv_arch`定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

----
![VGG11网络](../img/chapter05/vgg11.png)

----
<center>VGG11</center>

现在我们构造一个VGG网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为VGG-11。

In [5]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
in_channels = 1
fc_features = 512 * 7 * 7 # 根据卷积层的输出算出来的
fc_hidden_units = 4096 # 全连接层

下面我们实现VGG-11。

In [14]:
def vgg(conv_arch, in_channels, fc_features, fc_hidden_units):
    net = nn.Sequential()
    # 初始化
    in_channels = in_channels
    # 卷积层部分
    for i, (num_convs, out_channels) in enumerate(conv_arch):
        net.add_module('sequential' + str(i+1), vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    # 全连接层部分
    net.add_module( 'flatten', nn.Flatten() ) # 先把卷积层输出的形状匹配全连接层输入的形状
    net.add_module( 'dense0', nn.Sequential( nn.Linear(fc_features, fc_hidden_units), nn.ReLU() ) )
    net.add_module( 'dropout0', nn.Dropout(0.5) )
    net.add_module( 'dense1', nn.Sequential( nn.Linear(fc_hidden_units, fc_hidden_units), nn.ReLU() ) )
    net.add_module( 'dropout1', nn.Dropout(0.5) )  
    net.add_module( 'dense2', nn.Sequential( nn.Linear(fc_hidden_units, 10) ) )
                                 
    return net

net = vgg(conv_arch, in_channels, fc_features, fc_hidden_units)

In [15]:
net

Sequential(
  (sequential1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0,

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [16]:
X = torch.rand(1, 1, 224, 224)

# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

sequential1 output shape:  torch.Size([1, 64, 112, 112])
sequential2 output shape:  torch.Size([1, 128, 56, 56])
sequential3 output shape:  torch.Size([1, 256, 28, 28])
sequential4 output shape:  torch.Size([1, 512, 14, 14])
sequential5 output shape:  torch.Size([1, 512, 7, 7])
flatten output shape:  torch.Size([1, 25088])
dense0 output shape:  torch.Size([1, 4096])
dropout0 output shape:  torch.Size([1, 4096])
dense1 output shape:  torch.Size([1, 4096])
dropout1 output shape:  torch.Size([1, 4096])
dense2 output shape:  torch.Size([1, 10])


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

## 5.7.3 获取数据和训练模型

因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练。

In [10]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
small_conv_arch2 = [(1, 1, 64//ratio), (1, 64//ratio, 128//ratio), (2, 128//ratio, 256//ratio), 
                   (2, 256//ratio, 512//ratio), (2, 512//ratio, 512//ratio)]
print(small_conv_arch, small_conv_arch2)
net = vgg(small_conv_arch, 1, fc_features // ratio, fc_hidden_units // ratio)
print(net)

[(1, 16), (1, 32), (2, 64), (2, 128), (2, 128)] [(1, 1, 16), (1, 16, 32), (2, 32, 64), (2, 64, 128), (2, 128, 128)]
Sequential(
  (sequential1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (sequential4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size

除了使用了稍大些的学习率，模型训练过程与上一节的AlexNet中的类似。

In [13]:
device = device = d2l.get_current_device()
print(device)

cpu


In [7]:
lr, num_epochs, batch_size = 0.001, 5, 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.6411, train acc 0.759, test acc 0.877, time 35.6 sec
epoch 2, loss 0.1642, train acc 0.881, test acc 0.892, time 30.7 sec
epoch 3, loss 0.0937, train acc 0.898, test acc 0.908, time 30.5 sec
epoch 4, loss 0.0620, train acc 0.909, test acc 0.914, time 30.7 sec
epoch 5, loss 0.0457, train acc 0.916, test acc 0.913, time 30.6 sec


## 小结

* VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。

## 练习

* 与AlexNet相比，VGG通常计算慢很多，也需要更多的内存或显存。试分析原因。
* 尝试将Fashion-MNIST中图像的高和宽由224改为96。这在实验中有哪些影响？
* 参考VGG论文里的表1来构造VGG其他常用模型，如VGG-16和VGG-19 [1]。



## 参考文献

[1] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556.

## 扫码直达[知乎专栏](https://zhuanlan.zhihu.com/unicom-d2l)

![](../img/zhihu.png)